## Preprocessing

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

def reconstruct(pix_str, size=(48,48)):
    pix_arr = np.array(map(int, pix_str.split()))
    return pix_arr.reshape(size)

def save_data(X_train, X_test, y_train, y_test, fname='', folder='data/'):
    np.save(folder + 'X_train' + fname, X_train)
    np.save(folder + 'y_train' + fname, y_train)
    np.save(folder + 'X_test' + fname, X_test)
    np.save(folder + 'y_test' + fname, y_test)

def preprocess_data(filepath='data/fer2013.csv', split_ratio=0.1):
    
    print 'Reading csv into pandas dataframe'
    df = pd.read_csv(filepath)
    
    print 'Dropping Usage column'
    _Y = df.emotion.replace(0,1)
    _X = df.pixels
    frames = [_X, _Y]
    XY = pd.concat(frames, axis=1)
    
    print ' Uniqur values',_Y.unique()

    print 'Randomly shuffling the data'
    XY = XY.reindex(np.random.permutation(XY.index))
    
    print 'Reshaping 1x(48*48) array into a 48X48 2D matrix'
    XY['pixels'] = XY.pixels.apply(lambda x: reconstruct(x))

    print 'Converting Dataframe into numpy array'
    X = np.array([mat for mat in XY.pixels])
    X = X.reshape(-1, 1, X.shape[1], X.shape[2])
    Y = np.array([mat for mat in XY.emotion])

    print 'Generating train test splits'
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split_ratio)

    print 'Saving the files'
    save_data(X_train, X_test, y_train, y_test, fname='', folder='data/')

print 'Preprocessing begins'
# preprocess_data(filepath='data/fer2013.csv', split_ratio=0.1)
print 'Done!'

Preprocessing begins
Done!


## Deep Learning

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, RMSprop
from keras.utils.np_utils import to_categorical

import numpy as np
import sys

/home/dev/anaconda2/envs/p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# import dataset:
X_fname = 'data/X_train.npy'
y_fname = 'data/y_train.npy'
X_train = np.load(X_fname)
y_train = np.load(y_fname)
X_train = X_train.astype('float32')
y_train = np.delete(to_categorical(y_train), 0, 1)

In [4]:
# params:
batch_size = 128
nb_epoch = 100

# setup info:
print '\n------------------------ Training Parameters -------------------------'
print '                     X_train shape: ', X_train.shape # (n_sample, 1, 48, 48)
print '                     y_train shape: ', y_train.shape # (n_sample, n_categories)
print '                          img size: ', X_train.shape[2],'x', X_train.shape[3]
print '                        batch size: ', batch_size
print '                          nb_epoch: ', nb_epoch


------------------------ Training Parameters -------------------------
                     X_train shape:  (32298, 1, 48, 48)
                     y_train shape:  (32298, 6)
                          img size:  48 x 48
                        batch size:  128
                          nb_epoch:  100


In [5]:
# model architecture:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu',
                        input_shape=(1, X_train.shape[2], X_train.shape[3])))
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))

/home/dev/anaconda2/envs/p27/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(1, 48, 48...)`
  after removing the cwd from sys.path.
/home/dev/anaconda2/envs/p27/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  """
/home/dev/anaconda2/envs/p27/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
  
/home/dev/anaconda2/envs/p27/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(data_format="channels_first", pool_size=(2, 2))`
  import sys
/home/dev/anaconda2/envs/p27/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Co

In [6]:
# optimizer:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [7]:
print 'Training....'
model.fit(X_train, y_train, nb_epoch=nb_epoch, batch_size=batch_size,
          validation_split=0.1, shuffle=True, verbose=1)

Training....


/home/dev/anaconda2/envs/p27/lib/python2.7/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 29068 samples, validate on 3230 samples
Epoch 1/100
29068/29068 [==============================] - 17s 580us/step - loss: 1.7192 - acc: 0.2746 - val_loss: 1.6501 - val_acc: 0.3241
Epoch 2/100
29068/29068 [==============================] - 15s 528us/step - loss: 1.6146 - acc: 0.3436 - val_loss: 1.5607 - val_acc: 0.3669
Epoch 3/100
29068/29068 [==============================] - 15s 531us/step - loss: 1.5476 - acc: 0.3777 - val_loss: 1.5408 - val_acc: 0.3796
Epoch 4/100
29068/29068 [==============================] - 15s 532us/step - loss: 1.5115 - acc: 0.3966 - val_loss: 1.5010 - val_acc: 0.4077
Epoch 5/100
29068/29068 [==============================] - 15s 532us/step - loss: 1.4799 - acc: 0.4089 - val_loss: 1.4922 - val_acc: 0.4019
Epoch 6/100
29068/29068 [==============================] - 15s 532us/step - loss: 1.4522 - acc: 0.4208 - val_loss: 1.4743 - val_acc: 0.4152
Epoch 7/100
29068/29068 [==============================] - 16s 534us/step - loss: 1.4269 - acc: 0.4339 - val_lo

29068/29068 [==============================] - 15s 530us/step - loss: 0.3373 - acc: 0.8782 - val_loss: 3.4565 - val_acc: 0.4387
Epoch 59/100
29068/29068 [==============================] - 15s 524us/step - loss: 0.3283 - acc: 0.8801 - val_loss: 3.5569 - val_acc: 0.4390
Epoch 60/100
29068/29068 [==============================] - 15s 523us/step - loss: 0.3127 - acc: 0.8874 - val_loss: 3.3665 - val_acc: 0.4353
Epoch 61/100
29068/29068 [==============================] - 15s 525us/step - loss: 0.3317 - acc: 0.8810 - val_loss: 3.2173 - val_acc: 0.4433
Epoch 62/100
29068/29068 [==============================] - 16s 534us/step - loss: 0.2790 - acc: 0.8984 - val_loss: 3.6026 - val_acc: 0.4396
Epoch 63/100
29068/29068 [==============================] - 15s 522us/step - loss: 0.2677 - acc: 0.9039 - val_loss: 3.5771 - val_acc: 0.4492
Epoch 64/100
29068/29068 [==============================] - 16s 536us/step - loss: 0.2935 - acc: 0.8961 - val_loss: 3.6949 - val_acc: 0.4415
Epoch 65/100
29068/29068 [

In [8]:
# model result:
loss_and_metrics = model.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)
print 'Done!'
print 'Loss: ', loss_and_metrics[0]
print ' Acc: ', loss_and_metrics[1]

32298/32298 [==============================] - 5s 151us/step
Done!
Loss:  0.607635731933006
 Acc:  0.8901479967873682


In [11]:
model.save('saved_model.h5')

In [13]:
from keras.models import load_model
model2 = load_model('my_model.h5')

In [14]:
# model result:
loss_and_metrics = model2.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)
print 'Done!'
print 'Loss: ', loss_and_metrics[0]
print ' Acc: ', loss_and_metrics[1]

32298/32298 [==============================] - 5s 150us/step
Done!
Loss:  0.607635731933006
 Acc:  0.8901479967873682


In [ ]:
import cv2
import sys
import json
import time
import numpy as np
from keras.models import model_from_json


emotion_labels = ['angry', 'fear', 'happy', 'sad', 'surprise', 'neutral']
cascPath = sys.argv[1]

faceCascade = cv2.CascadeClassifier(cascPath)

# load json and create model arch
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights('model.h5')

def predict_emotion(face_image_gray): # a single cropped face
    resized_img = cv2.resize(face_image_gray, (48,48), interpolation = cv2.INTER_AREA)
    # cv2.imwrite(str(index)+'.png', resized_img)
    image = resized_img.reshape(1, 1, 48, 48)
    list_of_list = model.predict(image, batch_size=1, verbose=1)
    angry, fear, happy, sad, surprise, neutral = [prob for lst in list_of_list for prob in lst]
    return [angry, fear, happy, sad, surprise, neutral]

video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY,1)


    faces = faceCascade.detectMultiScale(
        img_gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.cv.CV_HAAR_SCALE_IMAGE
    )

    emotions = []
    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:

        face_image_gray = img_gray[y:y+h, x:x+w]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        angry, fear, happy, sad, surprise, neutral = predict_emotion(face_image_gray)
        with open('emotion.txt', 'a') as f:
            f.write('{},{},{},{},{},{},{}\n'.format(time.time(), angry, fear, happy, sad, surprise, neutral))


    # Display the resulting frame
    cv2.imshow('Video', frame)



    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()